##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys
def find_repo_root():
    try:
        script_path = os.path.abspath('.')
    except NameError:
        script_path = os.path.abspath(__file__)
    result = None
    while True:
        script_path = os.path.dirname(script_path)
        if os.path.isdir(os.path.join(script_path, '.git')):
            result = script_path
        if script_path == '/':
            return result
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)

## <i class="fa fa-wrench"></i> 安装必要依赖和工具

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [
    'wget',
    'iputils-ping'
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-folder-plus"></i> 创建Content文件夹
创建Content文件夹来还原Colab的运行环境，以减少代码总体修改量

In [ ]:
import os

def check_environment(output):
    environments = {
        "AutoDL": {
            "content_path": "/root/autodl-tmp/content",
            "env_name": "AutoDL"
        },
        "OpenBayes": {
            "content_path": "/openbayes/home/content",
            "env_name": "OpenBayes"
        }
    }
    # 检查输出是否包含字符串"autodl"或"openbayes"
    # for i in output:
    if "AutoDL" in output:
        return environments["AutoDL"]
    elif "OpenBayes" in output:
        return environments["OpenBayes"]
    # 如果输出中没有包含上述字符串，则返回空字典
    return {}


def detect_environment():
    # 初始化 content_path 和 env_name 变量
    content_path = None
    env_name = None

    # 将命令行存储在列表中
    commands = [
        "cd /openbayes/home && chmod +x /etc/welcome && /etc/welcome",
        "chmod +x /etc/autodl-motd && /etc/autodl-motd"
    ]

    # 遍历命令行列表，执行命令并存储输出
    for command in commands:
        try:
            import subprocess
            # 使用 subprocess 模块执行命令
            r = subprocess.run(command, shell=True, stdout=subprocess.PIPE)
            output = r.stdout
            # 将输出转换为字符串
            output = output.decode()
            # print(output)
            result = check_environment(output)
            content_path = result["content_path"]
            env_name = result["env_name"]
            # 如果检测到环境，则退出循环
            if content_path and env_name:
                break
        except Exception as e:
            # print("无法执行命令：", e)
            continue

    # 打印结果
    if content_path and env_name:
        print("当前运行环境：", env_name)
        print("内容路径：", content_path)
    else:
        print("未检测到当前运行环境")
        
    return {
        "content_path":content_path,
        "env_name":env_name
    }


content_path=detect_environment()["content_path"]
# 在根目录下创建Content文件夹
content_folder = content_path
if not os.path.exists(content_folder):
    os.makedirs(content_folder)

In [ ]:
!cd $content_path &&\
git clone https://github.com/YudongGuo/AD-NeRF

In [ ]:
!cd $content_path &&\
cd AD-NeRF &&\
conda env create -f environment.yml

In [ ]:
!cd $content_path &&\
source activate &&\
conda activate adnerf &&\
git clone https://github.com/facebookresearch/pytorch3d.git &&\
cd pytorch3d &&\
pip install -e .

## Put "01_MorphableModel.mat" to data_util/face_tracking/3DMM/; cd data_util/face_tracking; run

In [ ]:
!cd $content_path &&\
source activate &&\
cd AD-NeRF/data_util/face_tracking/ &&\
python convert_BFM.py

# 训练 AD-NeRF

In [ ]:
# 定义 id 名
id = 'Obama'

## 数据预处理

In [ ]:
!source activate &&\
conda activate adnerf &&\
cd $content_path &&\
cd AD-NeRF &&\
bash process_data.sh $id

## 训练两个 NeRF（Head-NeRF 和 Torso-NeRF）

### 使用命令训练 Head-NeRF

In [ ]:
!source activate &&\
conda activate adnerf &&\
cd $content_path &&\
cd AD-NeRF &&\
python NeRFs/HeadNeRF/run_nerf.py --config dataset/$id/HeadNeRF_config.txt

### 将最新的训练模型从 `dataset/$id/logs/$id_head` 复制到 `dataset/$id/logs/$id_com`

In [ ]:
id_head = id + '_head'
id_com = id + '_com'

!cd $content_path &&\
cd AD-NeRF &&\
cp -r dataset/$id/logs/$id_head/* dataset/$id/logs/$id_com/

### 使用命令训练 Torso-NeRF

In [ ]:
!source activate &&\
conda activate adnerf &&\
cd $content_path &&\
cd AD-NeRF &&\
python NeRFs/TorsoNeRF/run_nerf.py --config dataset/$id/TorsoNeRF_config.txt

# 运行 AD-NeRF 进行渲染

In [ ]:
# 用音频输入重建原始视频
!cd $content_path &&\
cd AD-NeRF &&\
source activate &&\
conda activate adnerf &&\
python NeRFs/TorsoNeRF/run_nerf.py --config dataset/$id/TorsoNeRFTest_config.txt --aud_file=dataset/$id/aud.npy --test_size=300

In [ ]:
# 用另一个音频输入驱动目标人物

!python NeRFs/TorsoNeRF/run_nerf.py --config dataset/$id/TorsoNeRFTest_config.txt --aud_file=${deepspeechfile.npy} --test_size=-1